In [1]:
import pandas as pd
import numpy as np
import gensim
import sklearn.feature_extraction as fe
import pickle
from gensim.models import KeyedVectors
from scipy.sparse import hstack, vstack, csr_matrix, coo_matrix

Using TensorFlow backend.


In [2]:
import os
path_to_data = os.getcwd() + "/data/"
path_to_w2v_models = "/Users/Nurislam/Documents/w2v_models/"

In [3]:
df = pd.read_csv(path_to_data + "data.tsv", sep="\t")
print (df.shape)
df = df.fillna("")
#df_without_nan = df.dropna()
#print (df_without_nan.shape)
#print (df.head())

(971157, 2)


In [4]:
texts = df[df.columns[1]].str.lower()
labels = df[df.columns[0]]

In [72]:
def write_texts():
    with open(path_to_data + "texts", "w") as f:
        for text in texts:
            f.write(text + "\n")
            
write_texts()

In [7]:
# stemming lemming

In [73]:
# doc2vec
# char2vec

class SentencesIterator(object):
    def __init__(self, corpus_dir, filenames):
        self.corpus_dir = corpus_dir
        self.filenames = filenames

    def __iter__(self):
        for f in self.filenames:
            for line in open(self.corpus_dir + f):
                yield line.split()
                
def create_w2v_own():
    sentences = SentencesIterator(path_to_data, ["texts"])

    # update parametres
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

    model.wv.save_word2vec_format(path_to_w2v_models + "/own_w2v_model", binary=True)
    
%time create_w2v_own()

CPU times: user 1min 27s, sys: 42.6 s, total: 2min 10s
Wall time: 1min 53s


In [74]:
def create_vectorizer(texts, use_if_idf = False):
    if use_if_idf:
        # изменить параметры
        v = fe.text.TfidfVectorizer()
    else:
        v = fe.text.CountVectorizer()

    v.fit_transform(texts)

    with open(path_to_data + "vectorizer", "wb") as f:
        pickle.dump(v, f)
        
%time create_vectorizer(texts, True)

CPU times: user 13.4 s, sys: 728 ms, total: 14.1 s
Wall time: 14.6 s


In [59]:
# ускорить вычисления
def get_vectors_by_texts_w2v_model(w2v, texts):
    arr = []
    
    num = 0

    # если отсутствует принимаем за zero или не учитываем?
    for i in texts:
        words = i.split(" ")
        words_exist = []
        for j in words:
            if j in w2v.vocab:
                words_exist.append(j)
                
        if len(words_exist) == 0:
            arr.append(np.zeros(w2v.vector_size))
        else:        
            vec = w2v[words_exist]
            arr.append(np.mean(vec, axis=0))
        if num % 100000 == 0:
            print ("chunk " + str(num))
            
        num += 1
        
    return np.array(arr)

In [60]:
# подбор моделей
exist_big_models = ["ruscorpora_russe.model.bin", #1 gb,
                    "ruwikiruscorpora_rusvectores2.bin", # 400,
                    "ruscorpora_1_600_2.bin", # 400,
                    "ruscorpora_1_300_10.bin"] # 200 NOUN, ADJ
    
w2v_big = KeyedVectors.load_word2vec_format(path_to_w2v_models + exist_big_models[0], binary=True)

%time vec_w2v_1 = get_vectors_by_texts_w2v_model(w2v_big, texts)

%time np.savetxt(path_to_data + "vec_w2v_1", vec_w2v_1, delimiter=',')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
CPU times: user 27.1 s, sys: 5 s, total: 32.1 s
Wall time: 34.9 s


In [64]:
w2v_own = KeyedVectors.load_word2vec_format(path_to_w2v_models + "/own_w2v_model", binary=True)

%time vec_w2v_2 = get_vectors_by_texts_w2v_model(w2v_own, texts)

%time np.savetxt(path_to_data + "vec_w2v_2", vec_w2v_2, delimiter=',')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
CPU times: user 30 s, sys: 1.16 s, total: 31.2 s
Wall time: 31.3 s


In [65]:
vectorizer = pickle.load(open(path_to_data + "vectorizer", "rb"))

vec_vectorizer = vectorizer.transform(texts)

with open(path_to_data + "vec_vectorizer", "wb") as f:
    pickle.dump(vec_vectorizer, f)

In [5]:
features = None
    
print ("load first")
with open(path_to_data + "vec_w2v_1", "rb") as f:
    vec_w2v_1 = np.loadtxt(path_to_data + "vec_w2v_1", delimiter=',')
print ("to csr")
vec_w2v_1 = csr_matrix(vec_w2v_1)
        
print ("load second")
with open(path_to_data + "vec_w2v_2", "rb") as f:
    vec_w2v_2 = np.loadtxt(path_to_data + "vec_w2v_2", delimiter=',')
print ("to csr")
vec_w2v_2 = csr_matrix(vec_w2v_2)

load first
to csr
load second
to csr


In [6]:
print ("load vectorizer")
with open(path_to_data + "vec_vectorizer", "rb") as f:
    vec_vectorizer = pickle.load(f)
    
print ("start hstack1")
features = hstack((vec_w2v_1, vec_w2v_2))
print (features.shape)
print ("start hstack 2")
features = hstack((features, vec_vectorizer))
print (features.shape)

load vectorizer
start hstack1
(971157, 400)
start hstack 2
(971157, 332388)


In [7]:
features = features.toarray()

In [8]:
%time np.savetxt(path_to_data + "features_full", features, delimiter=',')

OSError: [Errno 28] No space left on device

In [ ]:
def keras_perceptron_plain(X, y, model_id, epochs_main_neural, batch_size_main_neural):
    X = csr_matrix(X)
    y_train_new = []

    num_labels = max(y)+1 # исправить
    num_input = X[0].shape[1]

    for i in y:
        y_train_new.append(np.array([i]))

    y_train_new = np.array(y_train_new)

    one_hot_labels = keras.utils.to_categorical(y_train_new, num_classes=num_labels)

    tf.reset_default_graph()
    sess = tf.InteractiveSession()

    model = Sequential()
    # среднее между размером входа и размером выхода
    model.add(Dense((num_input + num_labels) / 2, activation='sigmoid', input_dim=num_input))

    # 17 - change
    model.add(Dense(num_labels, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=[metrics.mae, metrics.categorical_accuracy])

    model.fit(X.todense(), one_hot_labels, epochs=epochs_main_neural, batch_size=batch_size_main_neural)

    #PFTExtractingFs(model_id).save_keras_perceptron(model, "neural")

    return model

In [ ]:
keras_perceptron_plain(vec_vectorizer, labels, 30, 32)